# RAG application using open_source models (ollama)

In [15]:
from langchain_community.llms import Ollama 
from langchain_community.embeddings import OllamaEmbeddings

In [16]:
MODEL = "llama2"
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

In [10]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 

In [ ]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't answer the 
question, reply "I don't know."

context: {context}




Question: {question} 
"""

prompt = PromptTemplate.from_template(template)


In [14]:
chain = prompt | model | parser

In [ ]:
from langchain_community.document_loaders import PyPDFLoader 

file_name = "example.pdf"

loader = PyPDFLoader(file_name)
pages = loader.load_and_split()
pages

In [22]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

retriever = vectorstore.as_retriever()

In [11]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [23]:
questions = [
    #"Example question 1",
    #"Example question 2",
    #"Example question 3",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer : {chain.invoke({'question' : question})}")
    print()